In [3]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
      .appName("oms_returned_test")\
      .master("local[3]")\
      .config("spark.streaming.stopGracefullyOnShutdown", "true")\
      .getOrCreate()

In [5]:
spark

In [13]:
df = spark\
      .read\
      .format("json")\
      .load("C:/Users/PC/Documents/Jupyter/Udemy/Spark_Scala/Program/OmsDeliveryReturnedTest/data/trial.json")

In [12]:
dataStreamReader.show()

+------+--------------------+------+--------------------+--------------------+----------------+
|  code|  deliveryOrderLines|    id|        insertedDate|          receivedAt|returnedProducts|
+------+--------------------+------+--------------------+--------------------+----------------+
|string|[[string, string,...|string|2020-09-18T11:56:...|2021-06-07T14:47:39Z|   [[3, string]]|
+------+--------------------+------+--------------------+--------------------+----------------+



In [26]:
df_time = df.select("insertedDate")

In [30]:
df_time.show(1,False)

+------------------------+
|insertedDate            |
+------------------------+
|2020-09-18T11:56:28.783Z|
+------------------------+



In [34]:
to_timestamp(unix_timestamp(df_time.insertedDate,"yyyy-MM-dd'T'HH:mm"))

Column<b"to_timestamp(unix_timestamp(`insertedDate`, 'yyyy-MM-dd\\'T\\'HH:mm'))">

In [36]:
a = from_unixtime(unix_timestamp(from_utc_timestamp(to_timestamp(unix_timestamp(df_time.insertedDate , "yyyyy-MM-dd'T'HH:mm")*1000), 'Europe/Istanbul')),'yyyy-MM-dd')

In [38]:
print(a)

Column<b"from_unixtime(unix_timestamp(from_utc_timestamp(to_timestamp((unix_timestamp(`insertedDate`, 'yyyyy-MM-dd\\'T\\'HH:mm') * 1000)), Europe/Istanbul), yyyy-MM-dd HH:mm:ss), yyyy-MM-dd)">


In [81]:
 df = df_time\
      .withColumn("deliverycreateddate", expr("""from_unixtime(unix_timestamp(from_utc_timestamp(to_timestamp(unix_timestamp(insertedDate , "yyyy-MM-dd'T'HH:mm")), 'Europe/Istanbul')),'yyyy-MM-dd') """))\
      .drop("inserteddate")

In [82]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [83]:
df.show()

+-------------------+
|deliverycreateddate|
+-------------------+
|         2020-09-18|
+-------------------+

